# LLC4320 Examples

In [ ]:
import xarray as xr
import intake
import numpy as np
%matplotlib inline
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

### Load Data

Data is stored in [zarr](http://zarr.readthedocs.io) format on Google Cloud Storage.
This format is optimized for fast parallel access in the cloud.

In [ ]:
coords = intake.cat.LLC4320_grid.to_dask().reset_coords()
sst = intake.cat.LLC4320_SST.to_dask()
u = intake.cat.LLC4320_SSU.to_dask()
v = intake.cat.LLC4320_SSV.to_dask()
ds = xr.merge([sst, u, v])
ds

In [ ]:
print(f'Dataset Total Size: {ds.nbytes / 1e12:3.1f} TB')

The data is on a lat-lon-cap grid with 13 faces. To simplify things a bit, we just select one face from the North Atlantic.

In [ ]:
ds_natl = ds.sel(face=10)
coords_natl = coords.sel(face=10)
ds_natl

### Launch Dask Cluster

This allows us to parallelize calculations across cloud computing nodes.

In [ ]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client
cluster = KubeCluster(n_workers=10)
client = Client(cluster)
cluster

### Sea Surface Temperature

An interactive visualization.

In [ ]:
ocean_mask = coords_natl.hFacC.reset_coords(drop=True)>0
sst_natl = (ds_natl.SST.where(ocean_mask)
            .rename('SST'))
hv_image = hv.Dataset(sst_natl).to(hv.Image, kdims=['j', 'i'], dynamic=True)

%output holomap='scrubber' fps=1
%opts Image [width=800 height=500 invert_yaxis=True colorbar=True bgcolor='gray'] (cmap='RdBu_r')
regrid(hv_image, precompute=True)

### Use XGCM to Perform Calculus

[XGCM](https://xgcm.readthedocs.io/en/latest/) is a python packge for working with the datasets produced by numerical General Circulation Models (GCMs) and similar gridded datasets that are amenable to finite volume analysis. In these datasets, different variables are located at different positions with respect to a volume or area element (e.g. cell center, cell face, etc.) xgcm solves the problem of how to interpolate and difference these variables from one position to another.

In [ ]:
import xgcm
grid = xgcm.Grid(coords_natl.drop(['k', 'k_p1']), periodic=False)
grid

### Vorticity

An interactive map of relative vorticity.

In [ ]:
zeta = (-grid.diff(ds_natl.U * coords_natl.dxC, 'Y', boundary='extend')
        +grid.diff(ds_natl.V * coords_natl.dyC, 'X', boundary='extend'))/coords_natl.rAz
zeta

In [ ]:
vort_image = hv.Dataset(zeta.rename('vort')).to(hv.Image, kdims=['j_g', 'i_g'], dynamic=True)

%output holomap='scrubber' fps=0.25
%opts Image [width=800 height=500 invert_yaxis=True colorbar=True bgcolor='gray' logz=False] (cmap='RdBu_r')
regrid(vort_image, precompute=True).redim.range(vort=(-1e-4, 1e-4))

### Kinetic Energy

In [ ]:
eke = 0.5 * ( grid.interp(ds_natl.U, 'X', boundary='fill')**2 +
              grid.interp(ds_natl.V, 'Y', boundary='fill')**2
            ).where(ocean_mask).rename('EKE')
eke

In [ ]:
eke_image = hv.Dataset(np.log10(eke)).to(hv.Image, kdims=['j', 'i'], dynamic=True)

%output holomap='scrubber' fps=0.25
%opts Image [width=800 height=500 invert_yaxis=True colorbar=True bgcolor='gray' logz=False] (cmap='magma')
rg = regrid(eke_image, precompute=True).redim.range(EKE=(-4, 0))
rg

## Daily-Averaged Kinetic Energy

The data is resampled on the fly.

In [ ]:
ds_natl_daily = ds_natl.resample(time='D').mean()
ds_natl_daily

In [ ]:
eke_daily = 0.5 * (grid.interp(ds_natl_daily.U, 'X', boundary='fill')**2 +
                   grid.interp(ds_natl_daily.V, 'Y', boundary='fill')**2
                  ).where(ocean_mask).rename('EKE')
eke_daily

In [ ]:
eke_image = hv.Dataset(np.log10(eke_daily)).to(hv.Image, kdims=['j', 'i'], dynamic=True)

%output holomap='scrubber' fps=0.25
%opts Image [width=800 height=500 invert_yaxis=True colorbar=True bgcolor='gray' logz=False] (cmap='magma')
regrid(eke_image, precompute=True).redim.range(EKE=(-4, 0))

### Try to reproduce Peter Cornillon's Gulf Stream Calculation

In [ ]:
xc = coords_natl.XC[:, 0].load()
yc = coords_natl.YC[0].load()
eke_daily.coords['yc'] = yc.reset_coords(drop=True)
eke_daily.coords['xc'] = xc.reset_coords(drop=True)
eke_daily_swap = eke_daily.swap_dims({'i': 'yc', 'j': 'xc'})
eke_daily_swap

In [ ]:
eke_gs = eke_daily_swap.sel(yc=slice(40, 30)).sel(xc=[-70, -68, -66], method='nearest')
eke_gs

In [ ]:
# takes a very long time on a small cluster
eke_gs.isel(time=slice(0,30)).load()

In [ ]:
eke_gs.transpose().plot(col='xc', figsize=(12, 4))